In [1]:
import pandas as pd
import numpy as np

import math
from math import sin, cos, radians, pi

from shapely.geometry import Point
from shapely.geometry.polygon import Polygon

In [2]:
HOME="/home/spoudel/All_Thesis/Thesis_program/Data"
project="shawnee"
location="location_1"

to_save_final_in_home="final_data_to_train_on"

In [3]:
folder=HOME+"/"+project+"/"+location
location_in_time=pd.read_csv(folder+"/location_in_time.csv")
passes=location_in_time["passID"].values
each_data=[pd.read_csv(folder+f"/normalized_power_pass_{x}.csv") for x in location_in_time["passID"]]
# len(passes),each_data[0]

In [4]:
def compute_lat_long_passID(each_data,location_in_time):
    for i in range(len(passes)):
        print(i)
        each_data[i]["passID"]=i
        length=len(each_data[i])
        latitudes = np.linspace(location_in_time["lat_min"][i],location_in_time["lat_max"][i] , num=length)
        print(len(latitudes))
        each_data[i]["latitude"]=latitudes
        longitudes= np.linspace(location_in_time["long_min"][i],location_in_time["long_max"][i] , num=length)
        each_data[i]["longitude"]=longitudes
    return each_data

compute_lat_long_passID(each_data,location_in_time)
full_data=pd.concat(each_data)
full_data

0
358
1
360
2
366
3
366
4
360


,Unnamed: 0,timestamp,total_power,p0,p1,p2,p3,p4,p5,p6,passID,latitude,longitude
0,0,2023-01-16 17:24:39.120000+00:00,100.000000,-2.000000,2.000000,2.000000,2.000000,2.000000,-0.272938,2.000000,0,35.670006,-96.662316
1,1,2023-01-16 17:24:39.257000+00:00,30.908002,-2.000000,2.000000,1.456420,-0.565473,-0.569484,-0.296788,-0.267291,0,35.670008,-96.662316
2,2,2023-01-16 17:24:39.384000+00:00,35.629787,-2.000000,2.000000,0.855925,-0.607730,-0.779575,-0.053831,1.538686,0,35.670010,-96.662316
3,3,2023-01-16 17:24:39.511000+00:00,16.531542,-2.000000,1.243048,-0.668354,-0.669261,-0.469690,-1.085116,0.321046,0,35.670013,-96.662316
4,4,2023-01-16 17:24:39.638000+00:00,30.659669,-2.000000,0.839832,-0.430652,-0.938489,-0.585273,0.070700,1.146457,0,35.670015,-96.662316
...,...,...,...,...,...,...,...,...,...,...,...,...,...
355,355,2023-01-16 17:31:08.215000+00:00,18.406951,0.619154,2.000000,-0.067383,-0.711903,-0.299709,-0.395371,-0.997861,4,35.670775,-96.662319
356,356,2023-01-16 17:31:08.342000+00:00,67.584552,0.658804,0.218019,-0.212631,-0.182176,1.719183,1.880953,-0.183528,4,35.670778,-96.662319
357,357,2023-01-16 17:31:08.469000+00:00,82.175854,0.966712,0.201359,0.004933,2.000000,0.814174,-0.509651,0.454917,4,35.670780,-96.662319
358,358,2023-01-16 17:31:08.596000+00:00,100.000000,1.250349,-0.648439,1.299200,2.000000,-0.037076,-0.079023,1.638104,4,35.670782,-96.662319


In [5]:
def create_average_pass_count(data,min_x=None,max_x=None,min_y=None,max_y=None,roller_length=None):
    input_columns=data.columns
    data["prev_lat"]=data["latitude"].shift(1)
    data["prev_long"]=data["longitude"].shift(1)
    data["prev_pass"]=data["passID"].shift(1)

    #Constants
    if roller_length==None:
        rolller_length=0.0000115*0.3048*6

    #Hyper-parameters
    grid_size_x=0.000005*0.3048
    grid_size_y=0.000006*0.3048

    #Parameters
    if min_x==None:
        min_x=min_lat=data["latitude"].min()
    else:
        min_lat=min_x
    if max_x==None:
        max_x=max_lat=data["latitude"].max()
    else:
        max_lat=max_x
    if min_y==None:
        min_y=min_long=data["longitude"].min()
    else:
        min_long=min_y
    if max_y==None:
        max_y=max_long=data["longitude"].max()
    else:
        max_long=max_y
        
    print("min_x,max_x,min_y,max_y",min_x,max_x,min_y,max_y)
    
    print(max_x-min_x,max_y-min_y, "diff_x,diff_y")

    #create_matrix_grid
    print(min_x+(grid_size_x/2)-(rolller_length/2),max_x+(rolller_length/2),grid_size_x, "roller length", rolller_length)
    list_x=np.arange(min_x+(grid_size_x/2)-(rolller_length/2),max_x+(rolller_length/2),grid_size_x)
    list_y=np.arange(min_y+(grid_size_y/2)-(rolller_length/2),max_y+(rolller_length/2),grid_size_y)
    
#     print(list_x)

    data=data[data["passID"]==data["prev_pass"]]
    
    xv,yv=np.meshgrid(list_x,list_y)
    f_values=np.zeros(xv.shape)
    
    def point_pos(x0, y0, d, theta):
        theta_rad = radians(theta)
        return x0 + d*cos(theta_rad), y0 + d*sin(theta_rad)
    
    def given_two_point_find_rectangle(x0,y0,x1,y1):
        d=rolller_length/2
        theta=math.degrees(math.atan2((y1-y0),(x1-x0))) 
#         print("theta made with positive x axis=",theta)
        (xa,ya)=point_pos(x0,y0,d,theta-90)
        (xb,yb)=point_pos(x0,y0,d,theta+90)
        (xc,yc)=point_pos(x1,y1,d,theta+90)
        (xd,yd)=point_pos(x1,y1,d,theta-90)
        return (xa,ya),(xb,yb),(xc,yc),(xd,yd)
    
    def get_pass_values_and_update_f_value(rectangle_points):
        rectangle=Polygon(rectangle_points)
        pass_values=[]
        pass_location=[]
#         print(f_values)
        for ix in range(len(list_x)):
            for iy in range(len(list_y)):
                if rectangle.contains(Point(list_x[ix],list_y[iy])):
                    pass_location.append([ix,iy])
                    pass_values.append(f_values[iy,ix])
                    f_values[iy,ix]+=1
#         print("new line",pass_location,"\n",pass_values)
        return np.average(pass_values)+1
    
#     min_x,max_x,min_y,max_y,len(list_x),len(list_y),f.shape
    data["rect_points"]=data[["prev_lat","prev_long","latitude","longitude"]].apply(lambda x:
                                                                                         given_two_point_find_rectangle(x.prev_lat,x.prev_long,x.latitude,x.longitude),
                                                                                         axis=1
                                                                                        )
#     data["average_pass_count"]=data["rect_points"].apply(lambda x: get_pass_values_and_update_f_value(x))
    
    op_data_list=[]
    every_fifth_pass=[]
    for i, row in enumerate(data.itertuples(), 1):
        op_data_list.append(get_pass_values_and_update_f_value(row.rect_points))
    data["average_pass_count"]=op_data_list
    
    return data[np.append(input_columns,"average_pass_count")],xv,yv,f_values

In [6]:
data,xv,yv,f_values=create_average_pass_count(full_data)

min_x,max_x,min_y,max_y 35.67 35.6708 -96.662342 -96.662304
0.0007999999999981355 3.799999998932435e-05 diff_x,diff_y
35.6699902464 35.6708105156 1.5240000000000001e-06 roller length 2.1031200000000003e-05


/home/spoudel/anaconda3/envs/rica/lib/python3.7/site-packages/ipykernel_launcher.py:80: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/home/spoudel/anaconda3/envs/rica/lib/python3.7/site-packages/ipykernel_launcher.py:88: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [7]:
# np.append(np.array(data.columns),"average_pass_count")
data

,Unnamed: 0,timestamp,total_power,p0,p1,p2,p3,p4,p5,p6,passID,latitude,longitude,average_pass_count
1,1,2023-01-16 17:24:39.257000+00:00,30.908002,-2.000000,2.000000,1.456420,-0.565473,-0.569484,-0.296788,-0.267291,0,35.670008,-96.662316,1.000000
2,2,2023-01-16 17:24:39.384000+00:00,35.629787,-2.000000,2.000000,0.855925,-0.607730,-0.779575,-0.053831,1.538686,0,35.670010,-96.662316,1.000000
3,3,2023-01-16 17:24:39.511000+00:00,16.531542,-2.000000,1.243048,-0.668354,-0.669261,-0.469690,-1.085116,0.321046,0,35.670013,-96.662316,1.000000
4,4,2023-01-16 17:24:39.638000+00:00,30.659669,-2.000000,0.839832,-0.430652,-0.938489,-0.585273,0.070700,1.146457,0,35.670015,-96.662316,1.000000
5,5,2023-01-16 17:24:39.765000+00:00,26.623918,-1.905616,1.207969,0.279862,-0.889020,-0.640782,-0.274261,-0.467869,0,35.670017,-96.662316,1.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
355,355,2023-01-16 17:31:08.215000+00:00,18.406951,0.619154,2.000000,-0.067383,-0.711903,-0.299709,-0.395371,-0.997861,4,35.670775,-96.662319,2.666667
356,356,2023-01-16 17:31:08.342000+00:00,67.584552,0.658804,0.218019,-0.212631,-0.182176,1.719183,1.880953,-0.183528,4,35.670778,-96.662319,2.636364
357,357,2023-01-16 17:31:08.469000+00:00,82.175854,0.966712,0.201359,0.004933,2.000000,0.814174,-0.509651,0.454917,4,35.670780,-96.662319,2.750000
358,358,2023-01-16 17:31:08.596000+00:00,100.000000,1.250349,-0.648439,1.299200,2.000000,-0.037076,-0.079023,1.638104,4,35.670782,-96.662319,2.647059


In [8]:
data.to_csv(folder+"/"+to_save_final_in_home+"/data_pc"+project+"_"+location+".csv")